# Cross Validation and Simple Linear Regression

This is the process that gives us the internal and the cross-validation measures of <b> predictive accuracy </b>for a <b>simple linear regression</b>. The data are <b>randomly assigned</b> to a number of "folds", which in our context is the <b>test</b> and <b>training</b> folds. Each fold is removed, in turn, while the <b>remaining data</b> is used to <b>re-fit</b> the regression model and to <b>predict</b> at the <b>deleted observations.</b>

- We will predict employee salaries from different employee characteristics (or features). <br> 
- We are going to use a simple supervised learning technique: linear regression. 
- We want to build a simple model to determine how well Years Worked predicts an employee’s salary. Years Worked predicts an employee’s salary.

## 1. Importing

In [ ]:
# Importing libraries
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy import stats
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error as mse
from math import sqrt

In [ ]:
# Reading data into a dataframe

salary = pd.read_csv('data/salary.csv')
salary.head()

<h1><center>Type of data</center></h1> 

| Continuous | Categorical | Binary |
| --- | --- | --- |
| Salary | position | degree |
| exprior | field | otherqual |
| Yearsworked | - | male |
| market | - | - |
| yearsranked | - | -|
| yearsabs | - | - |

<h1><center>Feature description</center></h1> 

| Feature | Description |
| --- | --- |
| exprior | Years of experience prior to working in this field |
| Yearsworked | Years worked in this field |
| yearsrank | Years worked at current rank |
| market | Market value (1 = salary at market value for position, <br> <1 salary lower than market value for position,<br> >1 salary higher than market value for position) |
| degree | Has degree (0 = no 1= yes) |
| otherqual | Has other post-secondary qualification (0 = no, 1=yes) |
| position | Position (1 = Junior Employee 2=Manager 3= Executive) |
| male | 0 = no 1 1=yes |
| Field | Field of work (1 = Engineering 2=Finance 3=Human Resource 4=Marketing) |
| yearsabs |Years absent from work (e.g. due to illness / child rearing / personal reasons)|

<h1><center>Response description</center></h1> 

| Response | Description |
| --- | --- |
| <font color='black'> salary </font>| <font color='black'>  Annual salary in dollars </font> | 

## 2. Cleaning data
Data cleaning is the process of detecting and correcting corrupt or inaccurate data from a dataset, table, or database and refers to identifying incomplete, incorrect, inaccurate or irrelevant parts of the data and then replacing, modifying, or deleting the dirty or coarse data.
### 2.1 Examining missing values

In [ ]:
#Checking for missing values for our dataset

salary.isnull().sum()

In [ ]:
#Filling the missing values with the average
salary = salary.fillna(salary['salary'].mean())
salary.head()

##### What we did with the missing values in the data

- We <b>filled</b> the missing value that appeared in the <b>salary</b> column with the <b>mean</b> of that column.

### 2.2 Examing duplicates within the dataset

In [ ]:
#Dropping duplicates within the dataset
salary.drop_duplicates(inplace=True)

##### Dealing with duplicates within the dataset

We dropped any duplicates that may exist within the dataset

### 2.3 Examining outliers within the dataset
In statistics, an outlier is an observation point that is distant from other observations.


In [ ]:
#Checking for extreme values
sns.set()
sns.set(style="whitegrid")
fig, axes = plt.subplots(1,2, figsize=(15, 15))
sns.boxplot(x=salary["salary"], ax=axes[0], data = salary)
sns.boxplot(x=salary["yearsworked"], ax=axes[1],data = salary);


In [ ]:
#Removing the outliers
removed_outliers = salary['salary'].between(salary['salary'].quantile(.05), salary['salary'].quantile(.95))


##### Outliers

Using the box plot method have observed <b> two outliers</b> for the salary values, and <b>none</b> for the yearsworked values. 
How we intend on dealing with the outliers is to <b>remove </b> them, so that the model that we will build will not be affected.

## 3. Spliting data
These are two rather important concepts in data science and data analysis and are used as tools to prevent (or at least minimize) overfitting. We usually fit the model on a training set in order to make predications on a data that wasn’t trained (general data)

In [ ]:
#Split our data
x = salary[['exprior','yearsworked','yearsrank','market','degree','otherqual','position', 'male','Field','yearsabs']]
y = salary['salary']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)
y_train = pd.DataFrame(y_train)

### 3.1 Viewing the split data

In [ ]:
#showcasing the first 5 observations of the feature variables
X_train.head()


In [ ]:
#showcasing the first 5 observations of the response variable (salary)
y_train.head()

In [ ]:
#Describing the dataset of the response variable (salary)
y_train.describe()


In [ ]:
#Describing the dataset of the feature variables
X_train.describe()


### Describing the datasets
 - The training dataset contains <b> 80% of the overall dataset </b> and has been divided into two datasets the <b>feautures dataset </b> and the <b>response variable dataset</b>
 
 

## 4. Distribution of the data
The distribution of a statistical data set (or a population) is a listing or function showing all the possible values (or intervals) of the data and how often they occur.

### 4.1 Histograms of salaries and years worked 

In [ ]:
#Creating histograms with density line

sns.set()
sns.set(style="whitegrid")
fig, axes = plt.subplots(1, 2,figsize=(20, 6))
sns.distplot(X_train["yearsworked"], hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4}, ax=axes[0])

sns.distplot(y_train, hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4}, ax=axes[1])

axes[0].set_title('The histogram for years worked')
axes[1].set_title('The histogram for salary')
plt.show()

### 4.2 Scatterplot of salary and years worked

In [ ]:
#Creating scatterplot
combined = pd.concat([X_train, y_train], axis=1)
combined.head()
sns.regplot(x=combined['yearsworked'], y=combined['salary']);


##### Distribution of the salary and number of years worked
- Both graphs show a distribution that is not normal, they are both skewed to the right
- The salary distribution graph shows that there are more people earning between 35 000 dollars and 60 000 dollars and fewer people earning more than 75 000 dollars
- The number of years worked distribution graph shows that more people have worked within the years 0 to 26 years whereas the least have worked greater than 30 years

##### Scatter plot of the salary vs number of years worked

The <b> line of best fit </b> shows a <b>strong positive relationship</b> between years worked and the salary earned. which implies that when the number of years worked <b> increases </b> the salary inturn also <b> increase </b>

## 5. Modeling
### 5.1 Simple linear regression model using statsmodels 

In [ ]:
# Fit regression model 
results = smf.ols('salary ~ yearsworked', data=combined).fit()

# Inspect the results
print(results.summary())

### 5.2 Analysis of the model

##### 5.2.1 Does the model significantly predict the dependent variable? 

<b> According to the R-squared criterion:</b>
- Because there is only one feature variable modelled, we can either use the R-squared or the adjusted R-squared as the adjusted R-sqaured helps us with penalizing the R-squared for every variable inputed into the model.
- R-sqaured = 0.377 and adjusted R-squared = 0.376
  - So we can sonclude that the salary response variable is explained by 37% of the feauture variable which is number of years worked.
  

<b>According to the pvalue criterion: </b> <br>
H0: Does not have signficant relationship <br>
Ha: Has significant relationship
-  The pvalue given in the OLS is 0, and this is less than the significance level of 5%. Therefore we reject the null hypothesis stating that there is no evident relationship between the predictor and the response variable.



#####  5.2.2 What percentage of the variance in employees’ salaries is accounted for by the number of years they have worked
- The graph shows that about 37% of the data fits the regression model.

##### 5.2.3 Interpretation of coeficients

<b> Intercept: </b> 
- The coefficient for number of years worked is 40570
  - That is, if the number of years worked is 0, the base salary will be $40570 

<b> Number of years worked: </b> 
- The coefficient for number of years worked is 819.77
  - That is, if the number of years worked increased by one unit, the salary will then increase by $819.77



##### 5.2.4 Confidence intervals

- A 95% confidence interval is a range of values that you can be 95% certain that it contains the true mean of the population. Idealy, with regards to the interval,the true population mean value should be on that interval. If a confidence interval does not include a particular value, we can say that it is not likely that the particular value is the true population mean. However, even if a particular value is within the interval, we shouldn't conclude that the population mean equals that specific value.

- The confidence interval can also be used for coefficients of the regression model Use the confidence interval to assess the estimate of the population coefficient for each term in the model.


##### 5.2.4 What do the 95% confidence intervals [0.025, 0.975] mean?

You can be 95% confident that the confidence interval contains the value of the coefficient for the population.


### 5.3 Predictions 

In [ ]:
#Calculate expected salary for someone who worked for 12 years

year_12 = pd.DataFrame({'yearsworked':[12]})
prints = results.predict(year_12)
p=prints.iloc[0]
p


In [ ]:
# if combined.yearsworked == 12: return combined[combined.salary[i]] for i in combined

year_12 = pd.DataFrame(combined[combined.yearsworked == 12]['salary'])
year_12.sort_values(by=['salary'])

In [ ]:
#Calculate expected salary for someone who worked for 80 years
year_12 = pd.DataFrame({'yearsworked':[80]})
prints = results.predict(year_12)
p=prints.iloc[0]
p

In [ ]:
# if combined.yearsworked == 80: return combined[combined.salary[i]] for i in combined

year_80 = pd.DataFrame(combined[combined.yearsworked == 80]['salary'])
year_80.sort_values(by=['salary'])

#### Are there any problems with this prediction? If so, what are they?

- There was no problem in predicting the salary for the 12 years worked as 12 years is within the range of our dataset (Interpolation).
- The problem comes into play when we are predicting for the 80 years worked as this is not within the range (extrapolation)
- The disadvantage of predicting on extrapolated data is that it is unreliable if there are significant fluctuations in historical data. Assumes past trend will continue into the future – unlikely in many competitive business environments

## 6. Correlation
Correlation is a statistical technique that can show whether and how strongly pairs of variables are related.

### 6.1 Correlation Matrix



In [ ]:
rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(10, 10))
corr = combined.corr()
corr.style.background_gradient(cmap='coolwarm')

### 6.2 Correlation heat map

In [ ]:
correlation = combined.corr()

ax = sns.heatmap(
    correlation, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
salary_corr = combined[['salary', 'market', 'yearsworked', 'yearsrank']]

sns.pairplot(salary_corr)
plt.show()

### Interpreting correlation graph

- The above table shows the correlation between two variables, showcasing its <b>strengths and direction</b>. This helps us to select the features that have a significant impact on the response variable and therefore helping us predict the salaries.

- looking at the correlation between the response variable Salary and the feature variable yearsworked which is <b>0.623589</b>. This is a <b> fairly strong positive relationship </b>, and therefore we can assume as the number of years worked increases the value of salary also increases.

#### We have only looked at the number of years an employee has worked. What other employee characteristics might influence their salary?

Looking at the the correlation matrix, the features that have good correlation with salary are yearsrank, position and field. The rest of the features have less than 50% correlation with salary.

## 7. RSME
Root mean squared error (RMSE): RMSE is a quadratic scoring rule that also measures the average magnitude of the error. It’s the square root of the average squared differences between prediction and actual observation.


In [ ]:
# RSME for training dataset
train = results.predict(X_train['yearsworked'])

m_s_e = sqrt(mse(y_train, train))
m_s_e


In [ ]:
# RSME for test dataset
test = results.predict(X_test['yearsworked'])

msse = sqrt(mse(y_test, test))
msse

##### What is the difference in the Root Mean Square Error (RMSE) between the training and test sets? Is there any evidence of overfitting?

Our model performs better on the train dataset than it does on the test set. With this we can now conclude that the model is overfitted. We say this because our model shows low bias but high variance. To better train our model we need to identify relevant variables and terms that you are likely to influence our outcome that we hoping for

RMSE_train is bigger than  RMSE_test by slight margin therefore there is overfitting on our model.